In [1]:
import helpers as hlp
import mesa
import numpy as np
import pandas
# import seaborn
import random
%matplotlib inline
import matplotlib.pyplot as plt
# import bokeh
import math
# import pyvis
import uuid
import itertools
import scipy
import math

In [2]:
np.set_printoptions(
    linewidth=1000,
    threshold=np.inf,
    suppress=True
)

In [3]:
# parameter_dict = {
#             "ideology":0,
#             "economy":0,
#             "social":0,
#             "foreign":0,
#             "environment":0,
#             "authlib":0,
#             "party":0,
#             "fiscal":0,
#             "civil":0,
#             "nation":0,
#             "immigration":0,
#             "government":0,
#             "religion":0,
#             "healthcare":0
#         }

In [4]:
class LegisAgent(mesa.Agent):
    def __init__(self,model):
        super().__init__(model)
        
        self.parameter_dict = hlp.parameter_dict
        self.voters = None
        self.rand_seed = random.getstate()
        
        #norm_dist = scipy.stats.truncnorm(a=0,b=float('inf'))
        ##self.threshold = random.uniform(0,(np.sqrt(len(self.parameter_dict.values()))))
        # self.threshold = scipy.stats.truncnorm(a=0,b=float('inf'),loc=np.sqrt(len(self.parameter_dict.values()))).rvs(size=1)[0]
        # self.tolerance_threshold = scipy.stats.norm(loc=np.sqrt(len(self.parameter_dict.values()))).rvs(size=1)[0]
        # self.tolerance_threshold = abs(scipy.stats.norm().rvs(size=1)[0])
        
        self.tolerance_threshold = scipy.stats.uniform.rvs(size=1)[0]
        self.invitations = set()
        self.voted_bills = {}
    
    
    def step(self):
        #def alliance_phase(self):    
        def apply_threshold(self):
            rows, cols = np.indices(self.model.align_mat.shape)
            original_indicies = np.stack((rows,cols),axis=-1).reshape(-1,2)
            flat_array = self.model.align_mat.flatten()
            filtered_mask = flat_array >= self.threshold
            return {
                'array':flat_array[filtered_mask],
                'initial index':original_indicies[filtered_mask]
            }
    
    
    def session(self):
        
        def vote_on_bill(bills):
            ideology_subset = {key: self.ideology_dict[key] for key in bills[0].contents.keys()}
            voter_ideology_subset = {key: self.voters.ideology_dict[key] for key in bills[0].contents.keys()}
            agent_values = np.array(list(ideology_subset.values()))
            bill_values = np.array(list(bills[0].contents.values()))
            voter_values = np.array(list(voter_ideology_subset.values()))
            #alignment = np.linalg.norm(agent_values - bill_values)/len(bill_values)
            alignment = hlp.mean_absolute_difference(agent_values,bill_values)
            voter_alignment = hlp.mean_absolute_difference(voter_values,bill_values)
            
            if alignment < self.tolerance_threshold:
                bills[0].votes_for += 1
                self.voted_bills[bills[0]] = (alignment,1)
                self.voters.bill_directory[bills[0]] = (voter_alignment,1)
                #print(f'Legislator {self.unique_id} voted for {bills[0]} at {alignment} with a threshold of {self.tolerance_threshold}')
            else:
                bills[0].votes_against += 1
                self.voted_bills[bills[0]] = (alignment,0)
                self.voters.bill_directory[bills[0]] = (voter_alignment,0)
                #print(f'Legislator {self.unique_id} voted against {bills[0]} at {alignment} with a threshold of {self.tolerance_threshold}')
        
        vote_on_bill(self.model.pending_bills)
        
    def post_session(self):
        self.voters.adjust_legislator_performance()
        
        
        
class LegisParty:
    def __init__(self,unique_id):
        self.unique_id = unique_id
        self.party_ideology_metrics = None




class LegisModel(mesa.Model):
    
    def __init__(self,n,seed=None,attr={'alignmat_method':'norm'}):
        super().__init__(seed=seed)
        self.num_agents = n
        
        # Total voter population, initalized at 0 and will increase along with each LegisAgent
        self.electorate_size = 0
        self.step_number = 0
        self.voter_groups = []
        self.parties = []
        self.pending_bills = []
        self.passed_bills = []
        self.rejected_bills = []
        
        # Create agents
        for _ in range(self.num_agents):
            a = LegisAgent(self)
            a.ideology_dict = hlp.initialize_ideology(a.parameter_dict,mode='uniform')
            
            # voter_pop_modifier = lambda x: x**2
            # a.voters = scipy.stats.binom.rvs(voter_pop_modifier(self.num_agents),0.5,size=1)[0]
            
            tmp_voters = Voters(self,a)
            self.voter_groups.append(tmp_voters)
            
            self.electorate_size+= tmp_voters.population
        
        
        def generate_align_mat(method='norm'):
            
            calc_methods = {
                "norm": lambda x,y: np.linalg.norm(x - y),
                "mean-abs": lambda x,y: hlp.mean_absolute_difference(x,y)
            }
                    
            self.align_mat = np.eye(self.num_agents)
            itr = np.nditer(self.align_mat,order='K',flags=['multi_index'])
            for n in itr:
                i,j = itr.multi_index[0],itr.multi_index[1]
                agent_i = np.array(list(self.agents[i].ideology_dict.values()))
                agent_j = np.array(list(self.agents[i].ideology_dict.values()))
                self.align_mat[i,j] = calc_methods[method](agent_i,agent_j)
        generate_align_mat(method=attr['alignmat_method'])

     
     
    def alliance_phase(self,initialize_type='random',manual_override=None,method='equal'):
        
        def rand_initialize_parties(self,initialize_type,manual_override,method):
            
            rand_seed = random.getstate()
            if initialize_type == 'random':
                # Creates a random number of parties based on some bounding conditions
                num_parties = random.randrange(1,(self.num_agents//10)+1)
            elif initialize_type == 'manual':
                num_parties = manual_override
            for x in range(num_parties):
                self.parties.append(LegisParty(unique_id = uuid.uuid4()))
            match method:
                case 'equal':
                    # Randomly groups agents into (approximately) equal groups
                    sample_size = self.num_agents//num_parties
                    grouped_agents = hlp.generate_disjoint_sets(list(self.agents),sample_size)
                case 'random':
                    grouped_agents = hlp.assign_elements_to_bins(list(self.agents),num_parties)
            for i in range(len(self.parties)):
                self.parties[i].members = grouped_agents[i]
                
        if not self.parties:
            rand_initialize_parties(self,initialize_type,manual_override,method)
        
        
        
    def housekeeping_phase(self,distmat_method="norm"):
        
        def calculate_party_metrics(party):
    
            ideology_array = np.array([list(member.ideology_dict.values()) for member in party.members])
            avg_array = np.mean(ideology_array,axis=0)
            max_array = np.max(ideology_array,axis=0)
            min_array = np.min(ideology_array,axis=0)
            return dict(zip(hlp.parameter_dict.keys(),np.column_stack((avg_array,max_array,min_array))))
        
        for i in self.parties:
            i.party_ideology_metrics = calculate_party_metrics(i)
            
            
        def get_party_distmat(parties,base_dim=3,method="norm"):
            
            calc_methods = {
                "norm": lambda x,y: np.linalg.norm(x - y),
                "mean-abs": lambda x,y: hlp.mean_absolute_difference(x,y)
            }
            
            base_mat = np.array([list(i.party_ideology_metrics.values()) for i in parties])
            party_size = len(parties)
            id_dict_size = len(hlp.parameter_dict)
            idnt = np.zeros((party_size,party_size,id_dict_size,base_dim))
            for i,j,k in np.ndindex((party_size,party_size,id_dict_size)):
                idnt[:,:,k] = base_mat[:,k]
            for i,j,k,z in np.ndindex(idnt.shape):
                idnt[i,j,k,z] = calc_methods[method](idnt[i,j,k,z],idnt[j,i,k,z])
                        
                    # idnt[i,j,k,z] = hlp.mean_absolute_difference(idnt[i,j,k,z],idnt[j,i,k,z])
                
                # idnt[i,j,k,z] = np.linalg.norm(
                #     idnt[i,j,k,z] - idnt[j,i,k,z]
                # )
            return idnt
        self.party_distmat = get_party_distmat(self.parties,method=distmat_method)
        party_pairs = itertools.combinations(self.parties,2)
        
        
        
    def proposition_phase(self):
        Bill(unique_id=uuid.uuid4(),model=self)
        
    
    
    def step(self):
        # self.agents.shuffle_do('step')
        
        def file_bill(self):
            pending_bill = self.pending_bills[0]
            if pending_bill.votes_for > pending_bill.votes_against:
                pending_bill.active = True
                self.passed_bills.append(pending_bill)
                # print(f'{pending_bill} Passed at {pending_bill.votes_for} For to {pending_bill.votes_against} Against')
                self.pending_bills = self.pending_bills[1:]
            else:
                self.rejected_bills.append(pending_bill)
                # print(f'{pending_bill} Failed at {pending_bill.votes_for} For to {pending_bill.votes_against} Against')
                self.pending_bills = self.pending_bills[1:]
        
        # while len(self.pending_bills) > 0:
        while not not self.pending_bills:
            self.agents.shuffle_do('session')
            file_bill(self)
        
    def post_session(self):    
        self.agents.shuffle_do('post_session')

        
        
        
        
class Bill():
    def __init__(self,unique_id,model):
        
        self.unique_id = unique_id
        self.model = model
        model.pending_bills.append(self)
        self.num_issues = random.randrange(1,len(hlp.parameter_dict))
        issues = random.sample(list(hlp.parameter_dict.keys()),self.num_issues)
        #issue_values = scipy.stats.norm().rvs(size=self.num_issues)
        null_dict = {key:0 for key in issues}
        issue_values = list(hlp.initialize_ideology(null_dict,mode='uniform').values())
        self.contents = dict(zip(issues,issue_values))
        self.votes_for = 0
        self.votes_against = 0
        self.active = False
        
        
    def get_agent_alignments(self):
        
        alignment_list = []
        for i in self.model.agents:
            ideology_subset = {key: i.ideology_dict[key] for key in self.contents.keys()}
            agent_values = np.array(list(ideology_subset.values()))
            bill_values = np.array(list(self.contents.values()))
            alignment_list.append(np.linalg.norm(agent_values - bill_values))
        self.alignment_list = alignment_list
        
        
        
        
class Voters():
    def __init__(self,model,agent):
        voter_pop_modifier = lambda x: x**2
        self.legislator = agent
        agent.voters = self
        self.population = scipy.stats.binom.rvs(voter_pop_modifier(model.num_agents),0.5,size=1)[0]
        self.ideology_dict = hlp.initialize_ideology(hlp.parameter_dict,mode='uniform')
        self.legislator_performance = 1 - hlp.mean_absolute_difference(list(self.ideology_dict.values()),list(agent.ideology_dict.values()))
        self.bill_directory = {}
        ##(hlp.mean_absolute_difference(list(self.ideology_dict.values()),list(agent.ideology_dict.values())) / math.sqrt(self.population))
    
    def adjust_legislator_performance(self,threshold=0.5):      
        bill_val_array = np.array(list(self.bill_directory.values()))
        pos_array = bill_val_array[(bill_val_array[:, 0] < threshold) & (bill_val_array[:, 1] == 1) | ((bill_val_array[:, 0] > threshold) & (bill_val_array[:, 1] == 0))]
        neg_array = bill_val_array[(bill_val_array[:, 0] < threshold) & (bill_val_array[:, 1] == 0) | ((bill_val_array[:, 0] > threshold) & (bill_val_array[:, 1] == 1))]
        pos_wht_avg = (pos_array.shape[0]*pos_array[:,0].mean())/(pos_array.shape[0]+neg_array.shape[0])
        neg_wht_avg = (neg_array.shape[0]*neg_array[:,0].mean())/(pos_array.shape[0]+neg_array.shape[0])
        self.legislator_performance += (pos_wht_avg-neg_wht_avg)/np.sqrt(self.population)
        
    
    def vote(self):
        return np.count_nonzero(scipy.stats.bernoulli.rvs(self.legislator_performance,size=self.population))

In [5]:
attributes_dict = {
    'alignmat_method':'mean-abs'
}

my_model = LegisModel(250,attr=attributes_dict)

#TODO investigate performance slowdown of generating models with large n


In [6]:
my_model.alliance_phase()
my_model.housekeeping_phase(distmat_method="mean-abs")

ratio_list = []


for _ in range(1000):
    my_model.proposition_phase()
    my_model.step()
    #ratio_list.append(len(my_model.passed_bills)/(len(my_model.rejected_bills) if len(my_model.rejected_bills) > 0 else 1))

print(np.array(ratio_list).mean)

#print(my_model.pending_bills)

my_model.step()


#len(my_model.passed_bills), len(my_model.rejected_bills),len(my_model.passed_bills)/len(my_model.rejected_bills)

<built-in method mean of numpy.ndarray object at 0x000002C6DADAD710>


In [10]:
for i in my_model.agents:
    print(i.voters.vote()/i.voters.population)

0.5923818707810993
0.7447781722803314
0.5937360035830828
0.6109110608329881
0.7135861737878061
0.6287303833290455
0.7110122571766889
0.7205098568034418
0.705040828136051
0.5902768834513844
0.697994545162843
0.6631484392544352
0.6421106623068539
0.7729233916072741
0.6021667361133369
0.6543977825764656
0.8232725650178481
0.6806142888938349
0.6273865015005428
0.5725105099322871
0.5728965339518103
0.6699257067507156
0.6359107023090926
0.526689027311931
0.6118242053711469
0.6780533307600759
0.7370221199335123
0.6393094118404947
0.6070604078491728
0.7200408606269553
0.5731453958513121
0.6839130295147252
0.6440991019680276
0.7001026496439341
0.5924966417194396
0.7319705798070494
0.6851359574672518
0.6747107969151671
0.6332949013579298
0.6448681283762313
0.7075468674891844
0.6776044352759154
0.5513075246763236
0.6137768431784872
0.5193097254298178
0.6767202859696158
0.6697968078969296
0.6877845340804177
0.6490717960187877
0.639503248727715
0.6392012779552716
0.7992086724354232
0.65860978958459

In [7]:
# from bokeh.plotting import figure, show
# from bokeh.models import ColumnDataSource, HoverTool
# from bokeh.io import output_notebook

# output_notebook()

# source = ColumnDataSource(df)

# p = figure(height=800,width=800,title='Agent Disposition',x_axis_label='Ideology',y_axis_label='Authoritarian - Libertarian')
# p.scatter('ideology','economy',size=10,source=source)

# hover = HoverTool()
# hover.tooltips = [('unique_id','@unique_id')]
# p.add_tools(hover)

# show(p)

In [8]:
# import pandas as pd
# import numpy as np
# from bokeh.plotting import figure, show, output_notebook
# from bokeh.models import ColumnDataSource, Select, CustomJS, HoverTool
# from bokeh.layouts import column

# # Enable Bokeh output in the Jupyter Notebook
# output_notebook()

# # Convert the DataFrame to a dictionary
# df_dict = df.to_dict(orient='list')

# # Create a ColumnDataSource with initial data
# initial_x = 'ideology'
# initial_y = 'economy'
# source = ColumnDataSource(data={'x': df[initial_x], 'y': df[initial_y], 'unique_id': df['unique_id']})

# # Create a figure
# p = figure(title="Dynamic Plot with Pandas DataFrame", width=800, height=800)
# p.scatter('x', 'y', size=10, source=source)

# # Add HoverTool to display the 'unique_id' on hover
# hover = HoverTool(tooltips=[("unique_id", "@unique_id")])
# p.add_tools(hover)

# # Create Select widgets for choosing x and y columns
# x_select = Select(title="Select X-axis", value=initial_x, options=list(df.columns))
# y_select = Select(title="Select Y-axis", value=initial_y, options=list(df.columns))

# # JavaScript callback to update the plot based on column selection
# callback = CustomJS(args=dict(source=source, df=df_dict, x_select=x_select, y_select=y_select), code="""
#     const x_column = x_select.value;
#     const y_column = y_select.value;
#     const data = source.data;

#     // Update the data source using the dictionary passed from Python
#     data['x'] = df[x_column];
#     data['y'] = df[y_column];

#     // Emit changes to update the plot
#     source.change.emit();
# """)

# # Link the callback to the Select widgets
# x_select.js_on_change('value', callback)
# y_select.js_on_change('value', callback)

# # Display the layout with the Select widgets and the plot
# show(column(x_select, y_select, p))

Once bills are generated, the likelihood of each member to vote for the bill needs to be calculated based on the bill's contents and the ideological makeup of each member. 

The alignment of the bill with the ideology of each party needs to be calculated as well. Once this is determined it can be used to inform a key parameter associated with party pressure on each member's vote. (**new parameter needed**)

A class and a system for Voters to elect (or re-elect) members also needs to be determined. After this, a parameter (or parameters) associated with the pressure from voters and the need to be re-elected will also need to be implemented into the LegisAgent class.